In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib
import matplotlib.font_manager as fm
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt   # plots
import itertools
import numpy as np
import scipy.sparse as sp
from scipy.optimize import curve_fit
import time
import warnings

from liblibra_core import *
import util.libutil as comn
from libra_py import units
import libra_py.models.Holstein as Holstein
import libra_py.models.Tully as Tully
import libra_py.models.Subotnik as Subotnik
import libra_py.models.Esch_Levine as Esch_Levine
from libra_py import dynamics_plotting

import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

#from recipes import shxf, mqcxf, ehxf
#from recipes import ehrenfest_adi_ld, ehrenfest_adi_nac, ehrenfest_dia, mfsd
#from recipes import fssh, sdm, bcsh

import libra_py.dynamics.exact.compute as dvr
import libra_py.dynamics.exact.save as dvr_save

#from matplotlib.mlab import griddata
#%matplotlib inline 
warnings.filterwarnings('ignore')

## Color code
colors = {}
colors.update({"11": "#D20C0C", "12": "#FF5339", "13": "#FF8965"}) #red
colors.update({"21": "#00790C", "22": "#49A93F", "23": "#7DDB6D"}) #green
colors.update({"31": "#0009FF", "32": "#7145FF", "33": "#B176FF"}) #blue
colors.update({"41": "#FFA100", "42": "#FFD249", "43": "#FFFF7C"}) #orange
colors.update({"51": "#FF1DCE", "52": "#FFCDFF"}) #magenta
colors.update({"61": "#00ffea", "62": "#70FFFF", "63": "#AFFFFF"}) #cyan
colors.update({"71": "#bd20ff", "72": "#F461FF", "73": "#FF97FF"}) #indigo

clrs_index = ["11", "21", "31", "41", "51", "61", "71", "12", "22", "32", "42", "52", "62", "72" "13","23", "33", "53", "73"]

In [ ]:
plt.rc('axes', titlesize=40)      # fontsize of the axes title
plt.rc('axes', labelsize=40)      # fontsize of the x and y labels
plt.rc('legend', fontsize=40)     # legend fontsize
plt.rc('xtick', labelsize=40)     # fontsize of the tick labels
plt.rc('ytick', labelsize=40)     # fontsize of the tick labels

#plt.rc('figure.subplot', left=0.2)
#plt.rc('figure.subplot', right=0.95)
#plt.rc('figure.subplot', bottom=0.2)
plt.rc('figure.subplot', top=0.88)


font = {'family': 'serif',
        'color':  'blue',
        'weight': 'bold',
        'size': 36,
        }


## Populations

In [ ]:
# Error measure
def error(pop_ref, pop, dt = 1.0):
    """
    pop_ref, pop = nsteps x nstates 
    """
    
    nsteps, nstates = pop_ref.shape
    #diff = (1/nstates) * np.sum( np.array( np.abs(pop - pop_ref) ), axis=1 )
    diff = (1/nstates) * np.sum( np.array( np.abs(pop - pop_ref)**2 ), axis=1 )
    
    time_error = np.zeros( nsteps,)        
    tot_sum = 0.0
    for i in range(nsteps):          
        # Option 1
        #tot_sum =  ( i * tot_sum + diff[i] )/(i+1)        
        # Option 2
        tot_sum = tot_sum + diff[i] * dt
        time_error[i] = tot_sum 
    return diff, time_error

# Test example
A = np.array( [ [0.0, 0.0],  [0.0, 1.0],  [0.0, 2.0], [0.0, 3.0] ] )
B = np.array( [ [0.0, 1.0],  [0.0, 2.0],  [0.0, 3.0], [0.0, 4.0] ] )
t = np.array( [ 0, 1, 2, 3] )
X, err = error(A, B)
print(X, err)

In [ ]:
name_methods = ["SHXF, no BC", "MQCXF, no BC", "SHXF, BC", "MQCXF, BC"]
list_dirs = ["", "", "../", "../"]

In [ ]:
name_models = ["ECWR", "DAG", "SC Holstein", "DC Holstein", "Esch-Levine"]
nmodels = len(name_models)

In [ ]:
iconds = [[4], [5], [1], [3], [7]] # actually, these are just model indices
methods = [6, 7, 6, 7]

nmethods = len(methods)
niconds = len(iconds)

nsteps_icond = [4000, 3000, 8000, 8000, 8000, ]

dir_P="data_assessment_P/"
if not os.path.exists(dir_P):
    os.mkdir(dir_P)

A_se_adi = np.zeros( (nmethods, niconds,))
A_se_dia = np.zeros( (nmethods, niconds,))
A_sh_adi = np.zeros( (nmethods, niconds,))

for c, icond in enumerate(iconds):
    
    model_indx= icond[0]
    
    #fig, ax = plt.subplots(3, 3, figsize=(16, 12), sharey="row", sharex="col")
    
    #=================== Exact reference data =========================
    prf = F"../exact-model{model_indx}-icond0"        
    t, pops_adi_exact, pops_dia_exact = None, None, None
    
    with h5py.File(F"{prf}/data.hdf", 'r') as F:
        pops_adi_exact = np.array(F["pop_adi/data"][:nsteps_icond[c], :, 0])        
        pops_dia_exact = np.array(F["pop_dia/data"][:nsteps_icond[c], :, 0])         
        t = np.array(F['time/data'][:nsteps_icond[c]]) #* units.au2fs                   
    
    dt = t[1] - t[0]
        
    #=================== TSH data =========================
    for c2, method_indx in enumerate(methods):
        print(list_dirs[c2] + F"model_indx = {model_indx} method_indx = {method_indx}")
        prf = list_dirs[c2] + F"model{model_indx}-method{method_indx}-icond0"
 
        se_pops_adi, se_pops_dia, sh_pop_adi = None, None, None
        with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
            se_pops_adi = np.array(F['se_pop_adi/data'][:nsteps_icond[c],:]) 
            se_pops_dia = np.array(F['se_pop_dia/data'][:nsteps_icond[c],:]) 
            sh_pops_adi = np.array(F['sh_pop_adi/data'][:nsteps_icond[c],:]) 

            
        #=================== Now compute the errors =========================        
        err_se_adi, cum_err_se_adi = error(pops_adi_exact, se_pops_adi, dt)
        err_se_dia, cum_err_se_dia = error(pops_dia_exact, se_pops_dia, dt)
        err_sh_adi, cum_err_sh_adi = error(pops_adi_exact, sh_pops_adi, dt)
        
        A_se_adi[c2, c] = cum_err_se_adi[-1]/len(t)
        A_se_dia[c2, c] = cum_err_se_dia[-1]/len(t)
        A_sh_adi[c2, c] = cum_err_sh_adi[-1]/len(t)

np.savetxt(dir_P+"A_se_adi.txt", A_se_adi)
np.savetxt(dir_P+"A_se_dia.txt", A_se_dia)
np.savetxt(dir_P+"A_sh_adi.txt", A_sh_adi)

pA_se_adi = -np.log10(A_se_adi)
pA_se_dia = -np.log10(A_se_dia)
pA_sh_adi = -np.log10(A_sh_adi)

# Average
pA_se_adi_ave = np.average(pA_se_adi, axis=1)
pA_se_dia_ave = np.average(pA_se_dia, axis=1)
pA_sh_adi_ave = np.average(pA_sh_adi, axis=1)

#A_se_adi_ave = np.sum(A_se_adi, axis=1)
#A_se_dia_ave = np.sum(A_se_dia, axis=1)
#A_sh_adi_ave = np.sum(A_sh_adi, axis=1)
#
#pA_se_adi_ave = -np.log(A_se_adi_ave/np.sum(nsteps_icond))
#pA_se_dia_ave = -np.log(A_se_dia_ave/np.sum(nsteps_icond))
#pA_sh_adi_ave = -np.log(A_sh_adi_ave/np.sum(nsteps_icond))

In [ ]:
# SHXF
name_combined = ["no BC, SE", "no BC, SH", "BC, SE", "BC, SH"]

ind = np.array([4*x for x in np.arange(len(name_combined))])
ind = np.array([0, 3.8, 7.5, 11])
width = 0.5

fig, axs = plt.subplots(figsize=(14, 14))

axs.set_ylim([0, 5.2])

# SE pop
pA_combined_adi = np.zeros((4, nmodels))
pA_combined_adi[0, :] = pA_se_adi[0, :] # SE, no BC
pA_combined_adi[1, :] = pA_sh_adi[0, :] # SH, no BC
pA_combined_adi[2, :] = pA_se_adi[2, :] # SE, BC
pA_combined_adi[3, :] = pA_sh_adi[2, :] # SH, BC

pA_combined_adi_ave = np.zeros(4)
pA_combined_adi_ave[0] = pA_se_adi_ave[0]
pA_combined_adi_ave[1] = pA_sh_adi_ave[0]
pA_combined_adi_ave[2] = pA_se_adi_ave[2]
pA_combined_adi_ave[3] = pA_sh_adi_ave[2]

for imodel in range(nmodels):
    axs.bar(ind+width*imodel, pA_combined_adi[:, imodel], width, color = colors[clrs_index[imodel]], label=name_models[imodel])
axs.bar(ind+width*nmodels, pA_combined_adi_ave, width, color = "k", label="Average")

axs.set_xlim([-0.5,14])
axs.set_xlabel("Methods") 
axs.set_ylabel('Score') 
axs.set_title("Accuracy of populations, SHXF") 

axs.set_xticks(ind+width*2.5)
axs.set_xticklabels(tuple(name_combined)) 
#axs.legend(ncol=2, frameon=False, bbox_to_anchor = (-0.01, 1.01), loc="upper left")

In [ ]:
#MQCXF
name_combined = ["no BC", "BC"]

ind = np.array([4*x for x in np.arange(len(name_combined))])  
width = 0.5

fig, axs = plt.subplots(figsize=(7.5, 14))

axs.set_ylim([0, 5.2])

# SE pop
pA_combined_adi = np.zeros((2, nmodels))
pA_combined_adi[0, :] = pA_se_adi[1, :] # SE, no BC
pA_combined_adi[1, :] = pA_se_adi[3, :] # SE, BC

pA_combined_adi_ave = np.zeros(2)
pA_combined_adi_ave[0] = pA_se_adi_ave[1]
pA_combined_adi_ave[1] = pA_se_adi_ave[3]

for imodel in range(nmodels):
    axs.bar(ind+width*imodel, pA_combined_adi[:, imodel], width, color = colors[clrs_index[imodel]], label=name_models[imodel])
axs.bar(ind+width*nmodels, pA_combined_adi_ave, width, color = "k", label="Average")

axs.set_xlabel("Methods") 
axs.set_ylabel('Score') 
axs.set_title("Accuracy of populations, MQCXF") 

axs.set_xticks(ind+width*2.5)
axs.set_xticklabels(tuple(name_combined)) 
fig.savefig("tt.png", transparent=True, bbox_inches = "tight")
#axs.legend(ncol=1, frameon=False, loc="upper center")

## Coherence

In [ ]:
# Error measure
def errorC(coh_ref, coh, dt = 1.0):
    """
    coh_ref, coh = nsteps x nstates x nstates 
    """
    
    nsteps, nstates = coh.shape[0], coh.shape[1]
    coh_ref[0] = 0.0 # Remove inf values at the initial step.
    for istep in range(nsteps):
        np.fill_diagonal(coh_ref[istep], 0.0) 
    #diff = 1/(nstates*(nstates-1)) * np.sum(np.abs(coh_ref - coh), axis=(1,2))
    diff = 1/(nstates*(nstates-1)) * np.sum(np.abs(coh_ref - coh)**2, axis=(1,2))
    #print(coh_ref)
    
    time_error = np.zeros( nsteps,)        
    tot_sum = 0.0
    for i in range(nsteps):          
        # Option 1
        #tot_sum =  ( i * tot_sum + diff[i] )/(i+1)        
        # Option 2
        tot_sum = tot_sum + diff[i] * dt
        time_error[i] = tot_sum 
    return diff, time_error

## Test example
#A = np.array( [ [0.0, 0.0],  [0.0, 1.0],  [0.0, 2.0], [0.0, 3.0] ] )
#B = np.array( [ [0.0, 1.0],  [0.0, 2.0],  [0.0, 3.0], [0.0, 4.0] ] )
#t = np.array( [ 0, 1, 2, 3] )
#X, err = error(A, B)
#print(X, err)

In [ ]:
dir_coh="data_coherence/"

In [ ]:
iconds = [[4], [5], [1], [3], [7]] # actually, these are just model indices
nstates = [2,2,2,2,3]
methods = [6, 7, 6, 7]

nmethods = len(methods)
niconds = len(iconds)

nsteps_icond = [4000, 3000, 8000, 8000, 8000]

dir_coh="data_coherence/"

dir_C="data_assessment_C/"
if not os.path.exists(dir_C):
    os.mkdir(dir_C)

A_coh_adi = np.zeros( (nmethods, niconds,))

for c, icond in enumerate(iconds):
    
    model_indx= icond[0]
    #fig, ax = plt.subplots(3, 3, figsize=(16, 12), sharey="row", sharex="col")
    
    #=================== Exact reference data =========================
    prf = F"../exact-model{model_indx}-icond0"        
    t, cohs_adi_exact = None, None
    
    with h5py.File(F"{prf}/data.hdf", 'r') as F:
        cohs_adi_exact = np.array(F["coherence_adi/data"][:nsteps_icond[c], :, :])              
        t = np.array(F['time/data'][:nsteps_icond[c]]) #* units.au2fs

    dt = t[1] - t[0]
        
    #=================== TSH data =========================
    for c2, method_indx in enumerate(methods):
        print(list_dirs[c2] + F"model_indx = {model_indx} method_indx = {method_indx}")
        prf = list_dirs[c2] + F"model{model_indx}-method{method_indx}-icond0"
 
        se_cohs_adi = np.zeros((nsteps_icond[c], nstates[c], nstates[c]))
        with h5py.File(F"{prf}/mem_data.hdf", 'r') as F:
            for ist in range(nstates[c]):
                for jst in range(nstates[c]):
                    if ist <= jst: continue
                    rho2 = np.loadtxt(list_dirs[c2] + dir_coh + F"coherence_model{model_indx}_method{method_indx}_{ist}{jst}.txt")
                    se_cohs_adi[:, ist, jst] = rho2[:nsteps_icond[c]]
                    se_cohs_adi[:, jst, ist] = rho2[:nsteps_icond[c]]
                    #print(rho2)
        #=================== Now compute the errors =========================        
        err_se_adi, cum_err_se_adi = errorC(cohs_adi_exact, se_cohs_adi, dt)
        
        A_coh_adi[c2, c] = cum_err_se_adi[-1]/len(t)

np.savetxt(dir_C+"A_se_adi.txt", A_se_adi)

pA_coh_adi = -np.log10(A_coh_adi)

# Average
pA_coh_adi_ave = np.average(pA_coh_adi, axis=1)

In [ ]:
name_methods = ["no BC, SHXF", "no BC, MQCXF", "BC, SHXF", "BC, MQCXF"]
list_dirs = ["", "", "../", "../"]

ind = np.array([4*x for x in np.arange(len(name_methods))])  
ind = np.array([0, 4.3, 8.3, 12])
width = 0.4

fig, axs = plt.subplots(figsize=(18, 14), dpi=300)

axs.set_xlim([-0.5, 15])
axs.set_ylim([0, 5.2])

for imodel in range(nmodels):
    axs.bar(ind+width*imodel, pA_coh_adi[:, imodel], width, color = colors[clrs_index[imodel]], label=name_models[imodel])
axs.bar(ind+width*nmodels, pA_coh_adi_ave, width, color = "k", label="Average")

axs.set_xlabel("Methods") 
axs.set_ylabel("Score") 
axs.set_title("Accuracy of coherences") 

axs.set_xticks(ind+width*2.5)
axs.set_xticklabels(tuple(name_methods)) 
axs.legend(ncol=2, frameon=False, columnspacing=1, loc="upper center")